In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)
!pip install keras-tuner -q

2.15.0


In [3]:
fashion_mnist= keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)= fashion_mnist.load_data()

In [5]:
train_images= train_images/255
test_images= test_images/255

In [6]:
train_images[1].shape

(28, 28)

In [7]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
    model= keras.Sequential([
    keras.layers.Conv2D(
             hp.Int('Conv_1_filter', min_value=64, max_value= 128,step=16),
             kernel_size=hp.Choice('Conv_1_kernel',values=[3,5]),
             activation ='relu',
             input_shape=(28,28,1)
    ),

     keras.layers.Conv2D(
             hp.Int('Conv_2_filter', min_value=32, max_value= 64,step=16),
             kernel_size=hp.Choice('Conv_2_kernel',values=[3,5]),
             activation ='relu',

    ),
     keras.layers.Flatten(),
     keras.layers.Dense(
             hp.Int('dense_1_units', min_value=32, max_value= 128,step=16),
             activation ='relu',
     ),
      keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer= keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])


    return model

In [9]:
import keras_tuner
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search= RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name= 'Mnist Fashion')

Reloading Tuner from output\Mnist Fashion\tuner0.json


In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 04m 37s]
val_accuracy: 0.8705000281333923

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 122d 02h 12m 35s


In [13]:
model= tuner_search.get_best_models(num_models=1)[0]

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'output\Mnist Fashion\trial_0\checkpoint.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
model.summary()

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1,initial_epoch=3)